NSCLC-Radiogenomics .csv  

In [ ]:
import pandas as pd
df = pd.read_csv('C:/Users/sumay/Downloads/NSCLC.csv')

# Get a list of the variable/column names from the clinical dataset
column_names = df.columns.tolist()
print("Column names:")
for col in column_names:
    print(col)


TCGA-LIHC.tsv

In [ ]:
import pandas as pd
df = pd.read_csv('C:/Users/sumay/Downloads/TCGA-LIHC.tsv', sep='\t')

# Get a list of the variable/column names from the  dataset
column_names = df.columns.tolist()
print("Column names:")
for col2 in column_names:
    print(col2)



TCGA-LIHC.json

In [ ]:
import pandas as pd  # Import pandas for data manipulation
# Read the JSON file containing clinical data
# The file contains nested fields that need to be expanded

df = pd.read_json('C:/Users/sumay/Downloads/TCGA-LIHC-small.json')

# Explode the family_histories list so each entry becomes its own row
exploded = df.explode('family_histories')  # Each family_histories entry becomes a separate row

# Keep only rows where family_histories is a dictionary (not missing or malformed)
exploded = exploded[exploded['family_histories'].apply(lambda x: isinstance(x, dict))]

# Convert the dictionaries in family_histories to columns
family_histories_df = exploded['family_histories'].apply(pd.Series)  # Expand dict into columns

# Join with the original DataFrame's index or IDs
result = pd.concat([exploded.drop(columns=['family_histories']), family_histories_df], axis=1)  # Merge expanded columns

# Print expanded family_histories columns 
print("Family histories columns:")
for fam_col in family_histories_df.columns:
    print(fam_col)
print("End of family histories columns\n")

# Expand the nested 'demographic' column and print its columns 
# This column contains a dictionary for each patient
# We expand it to see all subfields

demographic_df = df['demographic'].apply(pd.Series)
print("Demographic columns:")
for dem_col in demographic_df.columns:
    print(dem_col)
print("End of demographic columns\n")

# Expand the nested 'project' column and print its columns 
if 'project' in df.columns:
    project_df = df['project'].apply(pd.Series)  # Expand project dict
    print("Project columns:")
    for proj_col in project_df.columns:
        print(proj_col)
    print("End of project columns\n")
else:
    project_df = None  # If not present, set to None

# Expand the nested 'diagnoses' column and print its columns
if 'diagnoses' in df.columns:
    # If diagnoses is a list of dicts, explode and expand
    if df['diagnoses'].apply(lambda x: isinstance(x, list)).any():
        diagnoses_exploded = df.explode('diagnoses')  # Each diagnosis becomes a row
        diagnoses_exploded = diagnoses_exploded[diagnoses_exploded['diagnoses'].apply(lambda x: isinstance(x, dict))]
        diagnoses_df = diagnoses_exploded['diagnoses'].apply(pd.Series)  # Expand dict into columns
    else:
        diagnoses_df = df['diagnoses'].apply(pd.Series)  # Expand single dict
    print("Diagnoses columns:")
    for diag_col in diagnoses_df.columns:
        print(diag_col)
    print("End of diagnoses columns\n")
else:
    diagnoses_df = None  # If not present, set to None

# Print other columns that do not need to be expanded
skip_cols = ['family_histories', 'demographic', 'project', 'diagnoses', 'follow_ups']  # Columns to skip
# List columns that are not nested
# These are simple fields like patient_id, etc.
display_columns = [col3 for col3 in df.columns if col3 not in skip_cols]
if display_columns:
    print("Other columns:")
    for col3 in display_columns:
        print(col3)
    print("End of other columns")

Saving .XLSX File

In [ ]:
import pandas as pd  # Import pandas for data manipulation
from pandas import ExcelWriter  # Import ExcelWriter to save DataFrames to Excel

# --- NSCLC CSV ---
df_csv = pd.read_csv('C:/Users/sumay/Downloads/NSCLC.csv')  # Load CSV file
columns_csv = pd.DataFrame({'Column Name': df_csv.columns.tolist()})  # Create DataFrame of column names

# --- TCGA-LIHC TSV ---
df_tsv = pd.read_csv('C:/Users/sumay/Downloads/TCGA-LIHC.tsv', sep='\t')  # Load TSV file
columns_tsv = pd.DataFrame({'Column Name': df_tsv.columns.tolist()})  # Create DataFrame of column names

# --- TCGA-LIHC JSON ---
df_json = pd.read_json('C:/Users/sumay/Downloads/TCGA-LIHC-small.json')  # Load JSON file

# Flatten nested columns for JSON using the same logic as your printout cell
json_columns = []  # List to store all expanded column names

# Expand family_histories
if 'family_histories' in df_json.columns:
    exploded = df_json.explode('family_histories')  # Each family_histories entry becomes a row
    exploded = exploded[exploded['family_histories'].apply(lambda x: isinstance(x, dict))]  # Keep only dicts
    family_histories_df = exploded['family_histories'].apply(pd.Series)  # Expand dict into columns
    json_columns += [f"family_histories.{col}" for col in family_histories_df.columns]  # Add expanded columns

# Expand demographic
if 'demographic' in df_json.columns:
    demographic_df = df_json['demographic'].apply(pd.Series)  # Expand demographic dict
    json_columns += [f"demographic.{col}" for col in demographic_df.columns]

# Expand project
if 'project' in df_json.columns:
    project_df = df_json['project'].apply(pd.Series)  # Expand project dict
    json_columns += [f"project.{col}" for col in project_df.columns]

# Expand diagnoses
if 'diagnoses' in df_json.columns:
    if df_json['diagnoses'].apply(lambda x: isinstance(x, list)).any():
        diagnoses_exploded = df_json.explode('diagnoses')  # Each diagnosis becomes a row
        diagnoses_exploded = diagnoses_exploded[diagnoses_exploded['diagnoses'].apply(lambda x: isinstance(x, dict))]  # Keep only dicts
        diagnoses_df = diagnoses_exploded['diagnoses'].apply(pd.Series)  # Expand dict into columns
    else:
        diagnoses_df = df_json['diagnoses'].apply(pd.Series)  # Expand single dict
    json_columns += [f"diagnoses.{col}" for col in diagnoses_df.columns]

# Add other columns (not nested)
skip_cols = ['family_histories', 'demographic', 'project', 'diagnoses', 'follow_ups']  # Columns to skip
other_cols = [col for col in df_json.columns if col not in skip_cols]  # Simple columns
json_columns += other_cols  # Add simple columns

columns_json = pd.DataFrame({'Column Name': json_columns})  # Create DataFrame of all JSON columns

# --- Save to Excel ---
output_path = 'C:/Users/sumay/Downloads/all_clinical_columns.xlsx'  # Output file path
with ExcelWriter(output_path) as writer:
    columns_csv.to_excel(writer, sheet_name='NSCLC_CSV', index=False)  # Save CSV columns
    columns_tsv.to_excel(writer, sheet_name='TCGA_LIHC_TSV', index=False)  # Save TSV columns
    columns_json.to_excel(writer, sheet_name='TCGA_LIHC_JSON', index=False)  # Save JSON columns
print(f"Excel file with all clinical columns saved to: {output_path}")  # Confirm save

Excel file with all clinical columns saved to: C:/Users/sumay/Downloads/all_clinical_columns.xlsx
